In [19]:
from forecasting.catchment_data import CatchmentData
import pickle
rebuild_catchment=False
if rebuild_catchment:
    catchment = CatchmentData("illinois-kerby", "14377100")

    pickle_out = open("data/catchment.pickle", "wb")
    pickle.dump(catchment, pickle_out)
    pickle_out.close()
else:
    pickle_in = open("data/catchment.pickle", "rb")
    catchment = pickle.load(pickle_in)

In [20]:
from forecasting.dataset import Dataset
ds = Dataset(catchment)

In [21]:
print("TRAIN:")
print(ds.merged_X_train.start_time())
print(ds.merged_X_train.end_time())

print()
print("VAL:")
print(ds.merged_X_validation.start_time())
print(ds.merged_X_validation.end_time())

print()
print("TEST:")
print(ds.merged_X_test.start_time())
print(ds.merged_X_test.end_time())

print()
print("CURRENT X:")
print(ds.merged_X_current.start_time())
print(ds.merged_X_current.end_time())

print()
print("CURRENT Y:")
print(ds.y_current.start_time())
print(ds.y_current.end_time())

TRAIN:
1988-10-01 01:00:00
2010-02-13 20:00:00

VAL:
2010-02-13 21:00:00
2015-06-19 07:00:00

TEST:
2015-06-19 08:00:00
2022-02-21 15:00:00

CURRENT X:
2022-03-21 19:00:00
2022-04-04 19:00:00

CURRENT Y:
2022-03-21 19:00:00
2022-03-31 17:00:00


In [22]:
X_trains = ds.X_trains[:2]
X_validations = ds.X_validations[:2]
X_tests = ds.X_tests[:2]

In [23]:
y_train = ds.y_train
y_validation = ds.y_validation

In [24]:
from darts.models import BlockRNNModel
pl_trainer_kwargs={
        "accelerator": "gpu",
        "gpus": [0]
    }

{'pl_trainer_kwargs': {'accelerator': 'gpu', 'gpus': [0]}, 
    'n_epochs': 10, 
    'input_chunk_length': 120, 
    'output_chunk_length': 96, 
    'model': 'GRU', 
    'hidden_size': 50, 
    'n_rnn_layers': 5, 
    'dropout': 0.01}

rnn1 = BlockRNNModel(input_chunk_length=120, output_chunk_length=24, model='GRU', n_epochs=10, hidden_size=50, n_rnn_layers=5, dropout=0.01, pl_trainer_kwargs=pl_trainer_kwargs)
rnn2 = BlockRNNModel(input_chunk_length=120, output_chunk_length=24, model='GRU', n_epochs=10, hidden_size=50, n_rnn_layers=5, dropout=0.01, pl_trainer_kwargs=pl_trainer_kwargs)


In [25]:
from darts_custom.regression_ensemble_model_custom import RegressionEnsembleModelCustom

ensemble = RegressionEnsembleModelCustom([rnn1, rnn2], regression_train_n_points=24)

In [26]:
ensemble.fit(series=y_train, past_covariates=X_trains)

True
[<TimeSeries (DataArray) (datetime: 187340, component: 11, sample: 1)>
array([[[0.59694288],
        [0.6       ],
        [0.        ],
        ...,
        [0.00830264],
        [0.59574147],
        [0.71039312]],

       [[0.57584697],
        [0.62222222],
        [0.        ],
        ...,
        [0.00830264],
        [0.59666436],
        [0.7102087 ]],

       [[0.54804684],
        [0.63333333],
        [0.        ],
        ...,
...
        ...,
        [0.260522  ],
        [0.29918532],
        [0.21160677]],

       [[0.43845535],
        [0.94444444],
        [0.        ],
        ...,
        [0.260522  ],
        [0.29941076],
        [0.21172419]],

       [[0.43845535],
        [0.94444444],
        [0.        ],
        ...,
        [0.260522  ],
        [0.2996362 ],
        [0.21184099]]])
Coordinates:
  * datetime   (datetime) datetime64[ns] 1988-10-01T01:00:00 ... 2010-02-13T2...
  * component  (component) object 'temp' 'humidity' ... 'temp_10d' 'temp_30d'


[2022-04-03 13:11:50,846] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 187173 samples.
[2022-04-03 13:11:50,846] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 187173 samples.
[2022-04-03 13:11:50,849] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 64-bits; casting model to float64.
[2022-04-03 13:11:50,849] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 64-bits; casting model to float64.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type       | Params
-----------------------------------------
0 | criterion | MSELoss    | 0     
1 | rnn       | GRU        | 70.8 K
2 | fc        | Sequential | 1.2 K 
-----------------------------------------
72.0 K    Trainable params
0         Non-trainable params
72.0 K    Total params
0.576    

Epoch 9: 100%|██████████| 5850/5850 [08:28<00:00, 11.50it/s, loss=9.41e-05, train_loss=0.000196]

[2022-04-03 14:36:11,674] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 187173 samples.
[2022-04-03 14:36:11,674] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 187173 samples.
[2022-04-03 14:36:11,678] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 64-bits; casting model to float64.
[2022-04-03 14:36:11,678] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 64-bits; casting model to float64.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type       | Params
-----------------------------------------
0 | criterion | MSELoss    | 0     
1 | rnn       | GRU        | 70.8 K
2 | fc        | Sequential | 1.2 K 
-----------------------------------------
72.0 K    Trainable params
0         Non-trainable params
72.0 K    Total params
0.576    


Epoch 9: 100%|██████████| 5850/5850 [08:41<00:00, 11.23it/s, loss=0.000317, train_loss=0.00137] 

...

[2022-04-03 16:01:53,114] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 187197 samples.
[2022-04-03 16:01:53,114] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 187197 samples.
[2022-04-03 16:01:53,116] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 64-bits; casting model to float64.
[2022-04-03 16:01:53,116] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 64-bits; casting model to float64.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type       | Params
-----------------------------------------
0 | criterion | MSELoss    | 0     
1 | rnn       | GRU        | 70.8 K
2 | fc        | Sequential | 1.2 K 
-----------------------------------------
72.0 K    Trainable params
0         Non-trainable params
72.0 K    Total params
0.576    


Predictions from emc make_multiple_predictions: 
[<TimeSeries (DataArray) (datetime: 24, component: 1, sample: 1)>
array([[[0.06313097]],

       [[0.06394203]],

       [[0.0648819 ]],

       [[0.06513418]],

       [[0.06525646]],

       [[0.06529938]],

       [[0.06590239]],

       [[0.06585288]],

       [[0.06584813]],

       [[0.06568321]],

...

       [[0.06637207]],

       [[0.06608826]],

       [[0.06629524]],

       [[0.06693188]],

       [[0.06771865]],

       [[0.06861525]],

       [[0.06986545]],

       [[0.07060254]],

       [[0.07158772]],

       [[0.07284002]]])
Coordinates:
  * datetime   (datetime) datetime64[ns] 2010-02-12T21:00:00 ... 2010-02-13T2...
  * component  (component) object 'level'
Dimensions without coordinates: sample, <TimeSeries (DataArray) (datetime: 24, component: 1, sample: 1)>
array([[[0.0627907 ]],

       [[0.06384619]],

       [[0.06466945]],

       [[0.06550981]],

       [[0.06678077]],

       [[0.06701874]],

       [[0.067

[2022-04-03 17:27:01,065] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 187197 samples.
[2022-04-03 17:27:01,065] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 187197 samples.
[2022-04-03 17:27:01,067] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 64-bits; casting model to float64.
[2022-04-03 17:27:01,067] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 64-bits; casting model to float64.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type       | Params
-----------------------------------------
0 | criterion | MSELoss    | 0     
1 | rnn       | GRU        | 70.8 K
2 | fc        | Sequential | 1.2 K 
-----------------------------------------
72.0 K    Trainable params
0         Non-trainable params
72.0 K    Total params
0.576     Total estimated model params size (MB)


Epoch 9: 100%|██████████| 5850/5850 [08:39<00:00, 11.27it/s, loss=0.000287, train_loss=9.44e-5] 


In [27]:
Xs = ds.Xs_current[:2]
y = ds.y_current

In [28]:
Xs[0].end_time()

Timestamp('2022-04-04 19:00:00', freq='H')

In [29]:
y.end_time()

Timestamp('2022-03-31 17:00:00', freq='H')

In [34]:
y_pred = ensemble.predict(n=24, series=y, past_covariates=Xs)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 5850it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



Predicting: 5850it [00:00, ?it/s]
Predictions from emc make_multiple_predictions: 
[<TimeSeries (DataArray) (datetime: 24, component: 1, sample: 1)>
array([[[0.01212708]],

       [[0.01193954]],

       [[0.01150848]],

       [[0.01150779]],

       [[0.01152571]],

       [[0.01097211]],

       [[0.01021618]],

       [[0.01074903]],

       [[0.01046049]],

       [[0.00959257]],

...

       [[0.00766698]],

       [[0.00708073]],

       [[0.00636368]],

       [[0.00604488]],

       [[0.0053028 ]],

       [[0.00521164]],

       [[0.00401389]],

       [[0.00423658]],

       [[0.00384878]],

       [[0.00362176]]])
Coordinates:
  * datetime   (datetime) datetime64[ns] 2022-03-31T18:00:00 ... 2022-04-01T1...
  * component  (component) object 'level'
Dimensions without coordinates: sample, <TimeSeries (DataArray) (datetime: 24, component: 1, sample: 1)>
array([[[0.01634385]],

       [[0.01547512]],

       [[0.01475678]],

       [[0.01420273]],

       [[0.01375861]],

    

In [35]:
target_scaler = ds.target_scaler

In [36]:
y_pred = target_scaler.inverse_transform(y_pred)

In [37]:
y_pred.pd_dataframe()

component,level
datetime,
2022-03-31 18:00:00,448.479393
2022-03-31 19:00:00,458.010600
2022-03-31 20:00:00,446.101997
2022-03-31 21:00:00,460.708071
2022-03-31 22:00:00,473.740998
2022-03-31 23:00:00,440.933299
2022-04-01 00:00:00,385.862924
2022-04-01 01:00:00,412.027953
2022-04-01 02:00:00,397.070670
